In [1]:
import datetime
import os

import backtrader
import numpy as np

os.chdir("../src")

from predictor import lstm

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error.

projects/harambe-6/locations/global/keyRings/harambe-6-dev/cryptoKeys/harambe-6-dev-key


In [4]:
# datafeed to provide indexes to frames to be used by a strategy
class FrameFeed(backtrader.feed.DataBase):
    params = (
        ("fromdate", datetime.datetime),
        ("todate", datetime.datetime),
        ("frames", None),
    )

    lines = ("datetime", "frame_index", "close")

    def __init__(self):
        super()

    def start(self):
        super(backtrader.feed.DataBase, self).start()
        # TODO: use assignment expressions
        self.frames = (
            frame for frame in enumerate(self.p.frames)
            if self.p.fromdate <= frame[1][-1][-1][0] <= self.p.todate
        )

    def _load(self):
        try:
            i, frame = next(self.frames)
        except StopIteration:
            return False

        self.lines.datetime[0] = backtrader.utils.date2num(frame[-1][-1][0])
        self.lines.frame_index[0] = i

        return True

In [3]:
# strategy to use by backtesting
class TestStrategy(backtrader.Strategy):
    params = (
        ("model", None),
        ("frames", [[[]]])
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt, txt))

    def __init__(self):
        self.closings = self.datas[0].close
        self.frame_index = self.datas[1].frame_index
        self.frames = self.params.frames
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        print(f"order size: {order.size}")
        
#         new_s, r, done, _ = env.step(a)
#         q_table[s, a] += r + lr * (y * np.max(q_table[new_s, :]) - q_table[s, a])
#         s = new_s

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED:')

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log("SELL EXECUTED:")

            print(f"Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}")
            print(self.broker.getvalue())

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        current_frame = [[self.closings[i]] for i in range(0, -15, -1)]
        previous_frame = [[self.closings[i]] for i in range(-1, -16, -1)]

        current_normalized_frame = lstm.normalize_frame(current_frame)
        previous_normalized_frame = lstm.normalize_frame(previous_frame)
    
        current_prediction = lstm.predict_sequences_multiple(self.p.model, [current_normalized_frame])
        previous_prediction = lstm.predict_sequences_multiple(self.p.model, [previous_normalized_frame])
        
        current_prediction_denorm = lstm.denormalize_dim(current_prediction[0][0], current_frame[0][0])
        # previous_prediction_denorm = lstm.denormalize_dim(previous_prediction[0][0], previous_frame[0][0])
        
        # print(f"current: {current_frame}, prediction: {current_prediction_denorm}")
        
        pred_diff = current_prediction[0][0] - previous_prediction[0][0]
        share_price = current_frame[-1][0]
        
        if not self.position:
            # can only hold, buy, or short
            if pred_diff > 0:
                if current_prediction_denorm > share_price:
                    self.log("BUY")
                    self.order = self.buy()
                else:
                    self.log("SHORT")
                    self.order = self.sell()
        else:
            # can only close or hold
            self.log("CLOSING")
            self.order = self.close()
        

#         if pred_diff > 0 and current_prediction_denorm > share_price:
#             if not self.position:
#                 self.log("BUY")
#                 self.order = self.buy()
                
#         elif pred_diff < 0 and current_prediction_denorm < share_price:
#             if not self.position:
#                 self.log("SHORT")
#                 self.order = self.sell()
                
#         elif self.position:
#             self.log("CLOSING")
#             self.order = self.close()
            
            
                

In [4]:
class Reverser(backtrader.sizers.FixedSize):

    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data)
        return cash // data.open[0] - 1
#         size = self.p.stake * (1 + (position.size != 0))
#         return size
#         if isbuy:
#             return cash // data.open[0] - 1
#         else:
#             return position.size

In [2]:
class PercentIncrease(backtrader.sizers.FixedSize):
    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data)
        if isbuy:
            return cash // 5 // data.open[0] - 1
        else:
            return position.size

In [9]:
SYMBOL="MSFT"

# get testing data
times = lstm.get_time_series_daily(SYMBOL, ["1. open"], outputsize="full")
vectors = lstm.times_to_vectors(times, include_time=True)[::-1]

train_vectors, test_vectors = lstm.partition_data(vectors, partition_coefficient=0.8)

train_frames = lstm.get_frames(train_vectors, 15, with_target=True)
test_frames = lstm.get_frames(test_vectors, 15, with_target=False)

train_no_dates = [[[col[1] for col in vector] for vector in frame] for frame in train_frames]
normalized_train = lstm.normalize_frames(train_no_dates)
x_train, y_train = lstm.seperate_xy(normalized_train)

In [10]:
# setup model
model = lstm.setup_lstm_model(x_train, y_train)

compilation time :  0.02097320556640625
Train on 4088 samples, validate on 216 samples
Epoch 1/1
4088/4088 [==============================] - 2s 473us/step - loss: 0.9509 - val_loss: 1.0949


In [11]:
# setup inital testing strategy
cerebro = backtrader.Cerebro()
cerebro.broker.setcash(100000.0)
# cerebro.addsizer(Reverser)
cerebro.addsizer(PercentIncrease)
cerebro.broker.setcommission(commission=0.001)

# add data and model to strategy
cerebro.addstrategy(TestStrategy, model=model, frames=test_frames)

from_date = datetime.datetime(2019, 1, 1)
to_date = datetime.datetime(2019, 6, 1)

cerebro.adddata(
    backtrader.feeds.YahooFinanceData(
        dataname=SYMBOL,
        fromdate=from_date,
        todate=to_date
    )
)

cerebro.adddata(
    FrameFeed(
        frames=test_frames,
        fromdate=from_date,
        todate=to_date
    )
)

print(f'Beginning Portfolio Value: {cerebro.broker.getvalue()}')

cerebro.run()

print(f'Final Portfolio Value: {cerebro.broker.getvalue()}')

Beginning Portfolio Value: 100000.0
2019-01-02, BUY
order size: 201.0
2019-01-03, BUY EXECUTED:
Price: 99.31, Cost: 19961.31, Comm: 19.96131
99441.35869
2019-01-03, CLOSING
order size: -201.0
2019-01-04, SELL EXECUTED:
Price: 98.93, Cost: 19961.31, Comm: 19.884930000000004
99883.77376000001
2019-01-22, SHORT
2019-01-22, SHORT
2019-01-24, BUY
order size: 187.0
2019-01-25, BUY EXECUTED:
Price: 106.39, Cost: 19894.93, Comm: 19.89493
99850.78883000002
2019-01-25, CLOSING
order size: -187.0
2019-01-28, SELL EXECUTED:
Price: 105.42, Cost: 19894.93, Comm: 19.713540000000002
99662.77529000002
2019-01-28, BUY
order size: 188.0
2019-01-29, BUY EXECUTED:
Price: 104.04999999999998, Cost: 19561.399999999998, Comm: 19.5614
99280.37389000002
2019-01-29, CLOSING
order size: -188.0
2019-01-30, SELL EXECUTED:
Price: 103.79, Cost: 19561.399999999998, Comm: 19.512520000000002
99574.82137000002
2019-01-31, BUY
order size: 192.0
2019-02-01, BUY EXECUTED:
Price: 102.96, Cost: 19768.32, Comm: 19.76832
99364.9